In [44]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
import torch.nn.functional as F

## 使用 MNIST 資料集  (載入資料)

In [45]:
train_dataset = dsets.MNIST(
    root = './data',
    train = True,
    transform = transforms.ToTensor(),
    download = True
)
test_dataset = dsets.MNIST(
    root = './data',
    train = False,
    transform = transforms.ToTensor()
)

In [46]:
print('train_dataset :', train_dataset.train_data.size())

train_dataset : torch.Size([60000, 28, 28])


In [47]:
print('train_dataset :', train_dataset.train_labels.size())

train_dataset : torch.Size([60000])


In [48]:
print('test_dataset :', test_dataset.test_data.size())

test_dataset : torch.Size([10000, 28, 28])


In [49]:
print('test_dataset :', test_dataset.test_labels.size())

test_dataset : torch.Size([10000])


In [70]:
batch_size = 100
n_iters = 1000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = False)

## Create Model !
![](./imgs/rnn.png)

In [77]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(RNNModel, self).__init__()   
        
        self.rnn = nn.RNN(
            input_size = input_dim,
            hidden_size = 100,
            num_layers = 1,
            batch_first=True,
            nonlinearity='relu')
        
        self.fc = nn.Linear(100, 10)
        
    def forward(self, x):
        # (layer_dim, batch_size, hidden_dim)
        h0 = None
        # another hidden state example :
        # h0 = Variable(torch.zeros(1, x.size(0), 100))
        
        out, hn = self.rnn(x, h0)
        # "out" dim : (100, 28, 100)
        # "-1" means the last time step
        
        out = self.fc(out[:, -1, :])  # (100, 100)
        # "out" dim : (100, 10)
        
        return out

In [72]:
input_dim = 28
output_dim = 10
model = RNNModel(input_dim, output_dim)

In [73]:
cost_fun = nn.CrossEntropyLoss()

In [74]:
lr = 0.1
opt = torch.optim.SGD(model.parameters(), lr = lr)

![](./imgs/rnn_weights.png)

In [75]:
iters = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # (100, 1, 28, 28) > (100, 28, 28)
        images = Variable(images.view(-1, 28, 28))
        labels = Variable(labels)
        
        opt.zero_grad()
        
        outputs = model(images)
        
        loss = cost_fun(outputs, labels)
        loss.backward()
        opt.step()
        
        iters += 1
        
        if iters % 5 == 0:
            correct = 0
            total = 0
            for images, labels in test_loader:
                images = Variable(images.view(-1, 28, 28))
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                
                correct += (predicted == labels).sum()
            acc = 100 * correct / total
            print('Iteration: {}. Loss: {}. Acc: {}'.format(iters, loss.data[0], acc))
                

Iteration: 5. Loss: 2.30782413482666. Acc: 8.02
Iteration: 10. Loss: 2.300601005554199. Acc: 9.71
Iteration: 15. Loss: 2.3077762126922607. Acc: 11.56
Iteration: 20. Loss: 2.301717758178711. Acc: 9.84
Iteration: 25. Loss: 2.304553270339966. Acc: 10.5
Iteration: 30. Loss: 2.303130626678467. Acc: 8.98
Iteration: 35. Loss: 2.3052358627319336. Acc: 9.28
Iteration: 40. Loss: 2.2994351387023926. Acc: 9.72
Iteration: 45. Loss: 2.302813768386841. Acc: 12.43
Iteration: 50. Loss: 2.291151762008667. Acc: 11.35
Iteration: 55. Loss: 2.296333074569702. Acc: 11.71
Iteration: 60. Loss: 2.296353816986084. Acc: 11.35
Iteration: 65. Loss: 2.291320323944092. Acc: 11.35
Iteration: 70. Loss: 2.301290512084961. Acc: 11.35
Iteration: 75. Loss: 2.298933744430542. Acc: 12.33
Iteration: 80. Loss: 2.2862627506256104. Acc: 11.37
Iteration: 85. Loss: 2.2919387817382812. Acc: 11.36
Iteration: 90. Loss: 2.281371593475342. Acc: 11.4
Iteration: 95. Loss: 2.295684337615967. Acc: 11.4
Iteration: 100. Loss: 2.2877917289733

Rerference :     
    - 莫煩 pytorch : https://morvanzhou.github.io/tutorials/machine-learning/torch/
    
    - 官方 tutorail : http://pytorch.org/tutorials/
    
    - https://blog.csdn.net/LiuPeiP_VIPL/article/details/75000742
    
    - https://www.youtube.com/watch?v=xCGidAeyS4M